# Análisis Exploratorio de Datos (EDA) - Limpieza y normalizacion

## Objetivo
Familiarizarse con los datos de transacciones y entender su estructura:
- Shape, tipos de datos, valores nulos, valores atípicos
- Distribuciones de montos y tipos de transacciones
- Identificación de patrones iniciales
- Limpiar las columnas y organziar el df

## Dataset
- **Archivos**: `sample_data_0006_part_00.parquet` y `sample_data_0007_part_00.parquet`
- **Ubicación**: `../data/`

In [43]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
from pathlib import Path

# Configuración para visualizaciones
plt.style.use('default')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 1. Carga de Datos

Vamos a cargar los dos archivos parquet y verificar si tienen la misma estructura para poder unirlos.

In [3]:
# Definir rutas de los archivos
data_path = Path("../data/")
file1 = data_path / "sample_data_0006_part_00.parquet"
file2 = data_path / "sample_data_0007_part_00.parquet"

# Cargar cada archivo por separado para explorar
print("📁 Cargando archivos...")
df1 = pd.read_parquet(file1)
df2 = pd.read_parquet(file2)

print(f"✅ Archivo 1 cargado: {file1.name}")
print(f"   Shape: {df1.shape}")
print(f"✅ Archivo 2 cargado: {file2.name}")
print(f"   Shape: {df2.shape}")

📁 Cargando archivos...
✅ Archivo 1 cargado: sample_data_0006_part_00.parquet
   Shape: (10758418, 8)
✅ Archivo 2 cargado: sample_data_0007_part_00.parquet
   Shape: (10758500, 8)


In [6]:
# Verificar estructura de ambos archivos
print("🔍 Verificando estructura de los archivos...\n")

print("📋 COLUMNAS ARCHIVO 1:")
print(f"Columnas: {list(df1.columns)}\n")

print("📋 COLUMNAS ARCHIVO 2:")
print(f"Columnas: {list(df2.columns)}\n")

# Verificar si las columnas son iguales
columnas_iguales = list(df1.columns) == list(df2.columns)
print(f"¿Las columnas son iguales? {columnas_iguales}")

if columnas_iguales:
    print("✅ Los archivos tienen la misma estructura, se pueden unir")
else:
    print("⚠️ Los archivos tienen diferente estructura")

🔍 Verificando estructura de los archivos...

📋 COLUMNAS ARCHIVO 1:
Columnas: ['merchant_id', '_id', 'subsidiary', 'transaction_date', 'account_number', 'user_id', 'transaction_amount', 'transaction_type']

📋 COLUMNAS ARCHIVO 2:
Columnas: ['merchant_id', '_id', 'subsidiary', 'transaction_date', 'account_number', 'user_id', 'transaction_amount', 'transaction_type']

¿Las columnas son iguales? True
✅ Los archivos tienen la misma estructura, se pueden unir


In [ ]:
# Unir los datasets
if columnas_iguales:
    print("🔗 Uniendo los datasets...")
    df = pd.concat([df1, df2], ignore_index=True)
    print(f"✅ Dataset unificado creado")
    print(f"   Shape final: {df.shape}")
    print(f"   Registros archivo 1: {len(df1)}")
    print(f"   Registros archivo 2: {len(df2)}")
    print(f"   Total registros: {len(df)}")
    
    # Liberar memoria de los dataframes individuales
    del df1, df2
else:
    print("❌ No se pueden unir los archivos debido a diferencias en estructura")

🔗 Uniendo los datasets...
✅ Dataset unificado creado
   Shape final: (21516918, 8)
   Registros archivo 1: 10758418
   Registros archivo 2: 10758500
   Total registros: 21516918


## 2. Analisis y preprocesamiento del dataset

In [10]:
# Vista rápida e información general del dataset
print("📊 INFORMACIÓN GENERAL DEL DATASET")
print("=" * 60)
print(f"💾 Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n📋 TIPOS DE DATOS:")
print("-" * 40)
tipo_counts = df.dtypes.value_counts()
for tipo, count in tipo_counts.items():
    print(f"  {tipo}: {count} columnas")

print(f"\n📝 NOMBRES DE LAS COLUMNAS:")
print("-" * 40)
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col} ({df[col].dtype})")

print(f"\nPRIMERAS 3 FILAS:")
print("-" * 40)
display(df.head(3))

print(f"\nÚLTIMAS 3 FILAS:")
print("-" * 40)
display(df.tail(3))

📊 INFORMACIÓN GENERAL DEL DATASET
💾 Memoria utilizada: 12234.13 MB

📋 TIPOS DE DATOS:
----------------------------------------
  object: 7 columnas
  datetime64[ns]: 1 columnas

📝 NOMBRES DE LAS COLUMNAS:
----------------------------------------
   1. merchant_id (object)
   2. _id (object)
   3. subsidiary (object)
   4. transaction_date (datetime64[ns])
   5. account_number (object)
   6. user_id (object)
   7. transaction_amount (object)
   8. transaction_type (object)

PRIMERAS 3 FILAS:
----------------------------------------


,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type
0,075d178871d8d48502bf1f54887e52fe,aa8dacff663072244d0a8ab6bbe36b93,824b2af470cbe6a65b15650e03b740fc,2021-09-12 18:32:03,648e257c9d74909a1f61c54b93a9e1b3,ba42d192a145583ba8e7bf04875f837f,178.33365037,CREDITO
1,075d178871d8d48502bf1f54887e52fe,a53bb81bd0bba2ae2535bda7ea5a550c,2d8d34be7509a6b1262336d036fdb324,2021-09-12 18:31:58,c0b62f9046c83ea5543ea46a497a4d6e,5cfff960ea6d732c1ba3e63d24f3be52,35.66673007,CREDITO
2,075d178871d8d48502bf1f54887e52fe,79f893ea65c06fe2933f3847c88c272f,5eeb18254850b21af0a6bb2697913cd3,2021-09-12 18:31:56,872d10143fc0ac7d5de467806f6bef81,c97e63a92c82c7217b333635d75928ed,142.66692029,CREDITO



ÚLTIMAS 3 FILAS:
----------------------------------------


,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type
21516915,838a8fa992a4aa2fb5a0cf8b15b63755,f19c70c14bae6f6c8d53c2a85f5f59f6,55fb0b01915693a7d5de686f9f4b1cb6,2021-11-30 11:01:07,90865ae9ab4d82a1feec13eee9a4a303,356058d6c9548879be6ef65a9733ea44,594.44550123,CREDITO
21516916,838a8fa992a4aa2fb5a0cf8b15b63755,8d43b7b55023ccae39735707a811f38f,55fb0b01915693a7d5de686f9f4b1cb6,2021-11-30 11:02:05,90865ae9ab4d82a1feec13eee9a4a303,356058d6c9548879be6ef65a9733ea44,172.38919535,CREDITO
21516917,838a8fa992a4aa2fb5a0cf8b15b63755,b15c20c071ab0df8fa15c71e4dd35e7d,189977b64fded8e9a37df9fc749fad5c,2021-11-29 14:07:30,528173dc0da13e78938c9e58c5cb3673,9b4a7ac86ed81ad330366e766c739808,47.55564009,CREDITO


In [11]:
# Estadísticas descriptivas básicas
print("📈 ESTADÍSTICAS DESCRIPTIVAS BÁSICAS")
print("=" * 60)

# Columnas categóricas/texto
text_cols = df.select_dtypes(include=['object', 'string']).columns
print(f"\n📝 Columnas de texto/categóricas encontradas: {len(text_cols)}")
if len(text_cols) > 0:
    print(f"Columnas: {list(text_cols)}")
    for col in text_cols:
        unique_count = df[col].nunique()
        print(f"\n  📊 {col}:")
        print(f"    Valores únicos: {unique_count:,}")
        print(f"    Tipo: {df[col].dtype}")
        if unique_count <= 20:
            print(f"    Valores: {sorted(df[col].unique())}")
        else:
            print(f"    Primeros 5 valores: {sorted(df[col].unique())[:5]}")
else:
    print("⚠️ No se encontraron columnas de texto")

📈 ESTADÍSTICAS DESCRIPTIVAS BÁSICAS
📊 Columnas numéricas encontradas: 0
⚠️ No se encontraron columnas numéricas

📝 Columnas de texto/categóricas encontradas: 7
Columnas: ['merchant_id', '_id', 'subsidiary', 'account_number', 'user_id', 'transaction_amount', 'transaction_type']

  📊 merchant_id:
    Valores únicos: 3
    Tipo: object
    Valores: ['075d178871d8d48502bf1f54887e52fe', '817d18cd3c31e40e9bff0566baae7758', '838a8fa992a4aa2fb5a0cf8b15b63755']

  📊 _id:
    Valores únicos: 21,516,901
    Tipo: object
    Primeros 5 valores: ['000000c00a3d728b6b5c1580b4816959', '000002ea806e97f97a6a3d5acd2855b2', '000003f2f33e6a45ce7483113c709aa1', '000004b9c3cb1548d4f9771458dc5582', '0000059697b8090d5079c6f3e475f68b']

  📊 subsidiary:
    Valores únicos: 16,833
    Tipo: object
    Primeros 5 valores: ['00015fd77a0f4d869bea31bb7244e375', '000b48cd239b8411eec69c69963cb5dd', '000ed83916ade2efe9c7ed9c81f6daa5', '0012d2042453597ebcf06140b0e4ec3f', '00130d1dfa59170a208840ea4004f423']

  📊 account_n

## Revision y transformacion de los tipos de datos de cada columna

In [17]:
#Transformar columna transaction_date a tipo datetime
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

# Transformar columna transaction_amount a tipo numérico
df['transaction_amount'] = pd.to_numeric(df['transaction_amount'], errors='coerce')

#Transformar transaction_type a tipo categórico
df['transaction_type'] = df['transaction_type'].astype('category')

#Verificar nuevamente los tipos de datos
print("\n📋 TIPOS DE DATOS DESPUÉS DE TRANSFORMACIONES:")
print("-" * 40)
print(df.dtypes)


📋 TIPOS DE DATOS DESPUÉS DE TRANSFORMACIONES:
----------------------------------------
merchant_id                   object
_id                           object
subsidiary                    object
transaction_date      datetime64[ns]
account_number                object
user_id                       object
transaction_amount           float64
transaction_type            category
dtype: object


## Verificar valores nulos

In [ ]:
print("\n🔍 VERIFICANDO VALORES NULOS")

nulos = df.isnull().sum()
print(f"Total de columnas con valores nulos: {nulos[nulos > 0].shape[0]}")



🔍 VERIFICANDO VALORES NULOS
Total de columnas con valores nulos: 0


## Verificar valores duplicados en IDs que deberían ser únicos (_id)

In [ ]:
print("\n🔍 VERIFICANDO VALORES DUPLICADOS EN IDs")

duplicados = df['_id'].duplicated().sum()
if duplicados > 0:
    print(f"⚠️ Se encontraron {duplicados} registros duplicados en la columna '_id'")
else:
    print("✅ No se encontraron registros duplicados en la columna '_id'")
        


🔍 VERIFICANDO VALORES DUPLICADOS EN IDs
⚠️ Se encontraron 17 registros duplicados en la columna '_id'


In [27]:
# Revisar los registros duplicados en '_id' manualmente

# 0. Numero de registros totales en la columna '_id'
total_ids = df['_id']
print(f"\nTotal de registros en la columna '_id': {len(total_ids)}")

# 1. Obtener la lista de IDs que aparecen más de una vez
dup_ids = df.loc[df['_id'].duplicated(), '_id'].unique()
print(f"IDs duplicados encontrados ({len(dup_ids)}):\n", dup_ids)

# 2. Filtrar todas las filas con esos IDs para inspección
df_dups = df[df['_id'].isin(dup_ids)].sort_values('_id')
display(df_dups)


Total de registros en la columna '_id': 21516901
IDs duplicados encontrados (0):
 []


,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type


In [26]:
# Despues de revisara a detalle No es un “doble pago” real, sino un retry o duplicación en la capa de ingestión del banco.
# Todas tienen la misma fecha y hora a excepción de una que tiene un segundo de diferencia.
# Por lo tanto, podemos eliminarlas sin problema el primer registro de cada ID duplicado.
print("\n🔄 ELIMINANDO REGISTROS DUPLICADOS EN '_id'")

df = df.drop_duplicates(subset=['_id'], keep='first').reset_index(drop=True)
print(f"✅ Duplicados de _id eliminados. Nuevo shape: {df.shape}")

# Verificar nuevamente los valores duplicados
duplicados = df['_id'].duplicated().sum()
print(f"Total de registros duplicados en '_id' después de limpieza: {duplicados}")


🔄 ELIMINANDO REGISTROS DUPLICADOS EN '_id'
✅ Duplicados de _id eliminados. Nuevo shape: (21516901, 8)
Total de registros duplicados en '_id' después de limpieza: 0


## Valores fuera de rango o invalidos

In [41]:
# 1. Montos <= 0
mask_amount_non_positive = df['transaction_amount'] <= 0
count_amount_non_positive = mask_amount_non_positive.sum()
pct_amount_non_positive = count_amount_non_positive / len(df) * 100
print(f"❌ Montos ≤ 0: {count_amount_non_positive} filas")

❌ Montos ≤ 0: 0 filas


In [40]:
# 2. Fechas fuera de rango

fecha_min = df['transaction_date'].min()
fecha_max = df['transaction_date'].max()
print(f"📅 Fecha mínima: {fecha_min}")
print(f"📅 Fecha máxima: {fecha_max}")
mask_date_before = df['transaction_date'] < fecha_min
mask_date_after  = df['transaction_date'] > fecha_max
count_date_bad = (mask_date_before | mask_date_after).sum()
pct_date_bad = count_date_bad / len(df) * 100
print(f"❌ Fechas fuera de [{fecha_min}] / [{fecha_max}]: {count_date_bad} filas")

📅 Fecha mínima: 2021-01-01 00:00:40
📅 Fecha máxima: 2021-11-30 23:59:49
❌ Fechas fuera de [2021-01-01 00:00:40] / [2021-11-30 23:59:49]: 0 filas


## IDs Inesperados

In [ ]:
# 3. IDs inesperados
# 4a) Longitud incorrecta (asumimos que debe ser 32 caracteres hex)
mask_id_length = df['_id'].str.len() != 32
count_id_length = mask_id_length.sum()
pct_id_length = count_id_length / len(df) * 100
print(f"❌ _id con longitud ≠ 32: {count_id_length} filas ({pct_id_length:.4f} %)")

# 4b) Caracteres no hexadecimales
hex_pattern = re.compile(r'^[0-9a-fA-F]{32}$')
mask_id_nonhex = ~df['_id'].str.match(hex_pattern)
count_id_nonhex = mask_id_nonhex.sum()
pct_id_nonhex = count_id_nonhex / len(df) * 100
print(f"❌ _id con caracteres no hex: {count_id_nonhex} filas ({pct_id_nonhex:.4f} %)")

# 4c) Espacios en blanco
mask_id_whitespace = df['_id'].str.contains(r'^\s|\s$')
count_id_whitespace = mask_id_whitespace.sum()
pct_id_whitespace = count_id_whitespace / len(df) * 100
print(f"❌ _id con espacios al inicio/final: {count_id_whitespace} filas ({pct_id_whitespace:.4f} %)")


❌ _id con longitud ≠ 32: 0 filas (0.0000 %)
❌ _id con caracteres no hex: 0 filas (0.0000 %)
❌ _id con espacios al inicio/final: 0 filas (0.0000 %)
✅ Espacios en _id recortados con .str.strip()
